# Chloride dynamics alter the input-output properties of neurons

## imports and settings

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from src.config import shared
from neuron import h

shared.INIT_NEURON(reinit=True)

# Figure 2

In [ ]:
from src.traces import figure_cli_distribution, figure_cli_heatmaps, figure_v_traces
# options
_distal_balanced_synapses = [(200, 30), (220, 90), (230, 140), (240, 200), (250, 260), (250, 300), (260, 400),
                             (270, 500), (280, 600), (290, 700), (300, 800)]
_proximal_balanced_synapses = [(260, 30), (290, 60), (330, 90), (370, 120), (430, 150), (490, 180), (530, 210),
                               (600, 240), (660, 270), (720, 300), (780, 330)]
# chosen
distal_balanced_synapses = [(200, 30), (230, 140), (250, 300), (270, 500), (300, 800)]
proximal_balanced_synapses = [(260, 30), (290, 60), (330, 90), (490, 180), (780, 330)]
mid_d = len(distal_balanced_synapses)//2
mid_p = len(proximal_balanced_synapses)//2
distal = [distal_balanced_synapses[0], distal_balanced_synapses[mid_d], distal_balanced_synapses[-1]]
proximal = [proximal_balanced_synapses[0], proximal_balanced_synapses[mid_p], proximal_balanced_synapses[-1]]


## Shape plot for $[Cl^-]_i$ along neuron (Figure 2A, 2D)

In [ ]:
figure_cli_distribution(distal[1], proximal[1])

## Heatmaps for output firing rate given 5 Hz balanced input
vary:
- number of $GABA_A$ synapses
- number of $NMDA+AMPA$ synapses

> if firing rate is above 15, skip further trials for this E:I combo and for larger E values (same I)




In [ ]:
from src.balanced_heatmap import run_NEURON, run_analysis

In [ ]:
# A geometric increase heatmap doesn't really work for a heatmap, which expects categorical data (pcolormesh is more suitable). 
geom_range = np.append(np.geomspace(10,100,2, dtype=int, endpoint=False), np.geomspace(100,1000,11, dtype=int))
# Use a consistent step for the heatmap.
exc_syn_list = inh_syn_list = np.arange(100, 600, 100) # narrow range
# default (from paper) is exc_syn_list = inh_syn_list = np.arange(0, 800 + 10, 10)
exc_syn_list

In [ ]:
result_files = run_NEURON(exc_syn_list=exc_syn_list, inh_syn_list=inh_syn_list, num_runs=2, max_num_runs=5)

In [ ]:
run_analysis(result_files)

![](results_balanced_heatmap/distal_KCC2_detail.png)
![](results_balanced_heatmap/proximal_KCC2_detail.png)

## Heatmaps for $[Cl^-]_i$ at different input frequencies (Figure 2C, 2F)

In [ ]:
figure_cli_heatmaps(distal, proximal, show_fr=True, savefig=True)

### Voltage traces for 5 Hz balanced input (Dynamic $Cl^-$)

In [ ]:
figure_v_traces(inh_region="distal", synapse_numbers=distal, 
                hz={'in': 5, 'ex': 5}, show_cli=True)
figure_v_traces(inh_region="proximal", synapse_numbers=proximal, 
                hz={'in': 5, 'ex': 5}, show_cli=True)

# Figure 3 

## Figure 3 voltage traces

### Voltage traces for 20 Hz balanced input (Dynamic $Cl^-$)

In [ ]:
from neuron import h
figure_v_traces(inh_region="distal", synapse_numbers=distal_balanced_synapses,
                hz={'in': 20, 'ex': 20}, show_cli=True)
figure_v_traces(inh_region="proximal", synapse_numbers=proximal_balanced_synapses,
                hz={'in': 20, 'ex': 20}, show_cli=True)

### Voltage traces for 20 Hz balanced input (Static $Cl^-$)

In [ ]:
# examples for static chloride
figure_v_traces(inh_region="distal", KCC2=False, synapse_numbers=distal_balanced_synapses,
                hz={'in': 20, 'ex': 20})
figure_v_traces(inh_region="proximal", KCC2=False, synapse_numbers=proximal_balanced_synapses,
                hz={'in': 20, 'ex': 20})

# Figure 5

In [ ]:
ifr_windowsize = 0.02  # in seconds
time_points = [int(ifr_windowsize*1000), 100, 500, 1000, 10000]
save_formats = ['png', 'eps']
common_args = dict(file_name="distal",
                   plot=False,
                   ifr_windowsize=ifr_windowsize,
                   synapse_type=1,  # 1 for persistent
                   synapse_numbers=(100, 100),
                   diam={'ldend': 1},
                   trials=1,
                   save=True)
static_runs, kcc2_runs = run_dynamic(0, 20, 0, 20, mid='half', fr_diff=10, **common_args)
static_runs_keys = static_runs.keys()
static_runs_keys = []
for static_name in static_runs_keys:
    file_name = "{save_location}traces_{result_name}.{format}".format(save_location=save_location,
                                                                      result_name=static_name, format="eps")
    if os.path.exists(file_name):
        continue
    static_run = static_runs[static_name]
    underscore = static_name.find("_")
    kcc2_run = kcc2_runs[static_name[:underscore] + "_KCC2" + static_name[underscore:]]
    f, axes = plot_compare_dcl(static_name, (static_run, kcc2_run),
                               ifr_windowsize=ifr_windowsize, combine=False, time_points=time_points)
    f.savefig(file_name)

f, axes = io_curve([static_runs, kcc2_runs], ifr_windowsize=ifr_windowsize, combine=True, heatmap=True, fill=True,
                   time_points=time_points, show_cli=True,
                   save_args=dict(fig_name="dynamic", formats=save_formats))

## Figure 5 voltage trace
- distal inhibition
- persistent synapses (synapse type 1)

In [ ]:
# trace for fig 5 heatmap
vm, cli = figure_v_traces(inh_region="distal", KCC2=True, show_cli=True, synapse_type=1,
                          synapse_numbers=[(100, 100)]*10, hz={'in': 4, 'ex': 12},
                          mean=False)


# Figure 3, 4, & 6 were done as follows:
1. run `python protocols.py <argument> <protocol>...`
    
    where `<argument>` is one of
    - `balanced` **for Figure 3**
    
    - `synapses` **for Figure 4**
    
    - `persistence` **for Figure 6A** 
        
        if `<protocol>` is provided, it can be any of
        - `pkcc2_homo` - **Figure 6D**
        - `duration` - **Figure 6D**
        - `dynamic_K` - **Figure 6D**
        - `pas` - **Figure 6D**
        - `diam`    - **Figure 6C**
        - `pkcc2`   - **Figure 6B**
        - `pcl` - *from earlier version - untested*
        - `cli` - *from earlier version - untested*
        
    
1. analysis using `ioAnalysis.m` (sorry)
    
    see `ioAnalysis` function for more details
    
**Figure 3A & B** input-output curves can also be plotted using python with the `balanced_iocurves.py` file